In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings

import argparse
import logging
import pandas as pd
import random
import os
import numpy as np
from loggers import logger

from process import input_data, preprocess

def set_logger(log_name):
    log_obj = logger.AutoMLLog(log_name)
    log_obj.set_handler('automl_process')
    log_obj.set_formats()
    auto_logger = log_obj.addOn()
    
    auto_logger.info('logger 세팅')

In [2]:
#데이터 인코딩 문제

In [3]:
log_name = 'practice'
set_logger(log_name)
data, var_list, num_var, obj_var = input_data.Data_load('storage/Walmart.csv', log_name).read_data()
df = preprocess.Preprocessing(log_name, data, var_list, num_var, obj_var, target='Weekly_Sales', date_var= 'Date', store_list=['Store'], unit='week').df

logger 세팅
csv 데이터 불러오기
storage/Walmart.csv
데이터 구성: 6435 행, 8열
Memory usage of dataframe is 0.39MB
Memory usage after optimization is: 0.10MB
Decreased by 73.7%
결측치 처리
결측치 처리 이후 데이터 구성: 6435 행, 8열
전처리를 위한 target, date, store 분리
정규화 진행
라벨 인코딩 진행
전처리 완료


   Temperature  Fuel_Price       CPI  Unemployment  Holiday_Flag  \
0    -0.995605   -1.712891  1.004883      0.058716     -0.274170   
1    -1.202148   -1.763672  1.007812      0.058716      3.646484   
2    -1.124023   -1.840820  1.007812      0.058716     -0.274170   
3    -0.761230   -1.738281  1.010742      0.058716     -0.274170   
4    -0.768066   -1.597656  1.010742      0.058716     -0.274170   

   Weekly_Sales        Date  Store  
0   1643690.875  05-02-2010      1  
1   1641957.500  12-02-2010      1  
2   1611968.125  19-02-2010      1  
3   1409727.625  26-02-2010      1  
4   1554806.625  05-03-2010      1  
시계열용 전처리 진행
시계열 전처리 후 df
   Temperature  Fuel_Price       CPI  Unemployment  Holiday_Flag  \
0     0.607910   -1.644

In [4]:
train_test = df[df['Store']=='1']

In [10]:
train_test['ds'] = train_test['Date']
train_test['y'] = train_test['Weekly_Sales']
train_test['cap'] = np.max(train_test['Weekly_Sales'])
train_test['floor'] = np.min(train_test['Weekly_Sales'])
train_test = train_test[['ds','y','cap','floor']]

C:\Users\LDCC\anaconda3\envs\practice\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\LDCC\anaconda3\envs\practice\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\LDCC\anaconda3\envs\practice\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

In [11]:
print(train_test.shape)
train_test.head()

(143, 4)


,ds,y,cap,floor
0,2010-01-10,1453329.500,2387950.25,1316899.25
1,2010-02-04,1594968.250,2387950.25,1316899.25
2,2010-02-07,1492418.125,2387950.25,1316899.25
3,2010-02-19,1611968.125,2387950.25,1316899.25
4,2010-02-26,1409727.625,2387950.25,1316899.25


In [7]:
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
import optuna

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [8]:
test_size = int(train_test.shape[0] / 10)
train_df = train_test.iloc[:-test_size, :]
val_df = train_test.iloc[-test_size:int(-test_size/2), :]
test_df = train_test.iloc[int(-test_size/2):, :]

In [9]:
train_df

,Temperature,Fuel_Price,CPI,Unemployment,Holiday_Flag,Weekly_Sales,Date,Store,time_idx,week,ds,y,cap,floor
0,0.607910,-1.644531,1.017578,-0.085022,-0.274170,1453329.500,2010-01-10,1,0,1,2010-01-10,1453329.500,2387950.25,1316899.25
1,0.087708,-1.393555,0.998047,-0.101624,-0.274170,1594968.250,2010-02-04,1,1,5,2010-02-04,1594968.250,2387950.25,1316899.25
2,1.098633,-1.500000,1.007812,-0.114136,-0.274170,1492418.125,2010-02-07,1,2,5,2010-02-07,1492418.125,2387950.25,1316899.25
3,-1.124023,-1.840820,1.007812,0.058716,-0.274170,1611968.125,2010-02-19,1,3,7,2010-02-19,1611968.125,2387950.25,1316899.25
4,-0.761230,-1.738281,1.010742,0.058716,-0.274170,1409727.625,2010-02-26,1,4,8,2010-02-26,1409727.625,2387950.25,1316899.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,1.312500,-0.159912,1.277344,-0.455811,-0.274170,1540421.500,2012-06-29,1,124,26,2012-06-29,1540421.500,2387950.25,1316899.25
125,1.261719,0.810059,1.293945,-0.582520,3.646484,1661767.375,2012-07-09,1,125,28,2012-07-09,1661767.375,2387950.25,1316899.25
126,0.892090,-0.223755,1.277344,-0.582520,-0.274170,1527014.000,2012-07-13,1,126,28,2012-07-13,1527014.000,2387950.25,1316899.25
127,1.071289,-0.104675,1.277344,-0.582520,-0.274170,1497954.750,2012-07-20,1,127,29,2012-07-20,1497954.750,2387950.25,1316899.25


In [12]:
def objective(trial):
    params = {
        'changepoint_prior_scale': trial.suggest_float('changepoint_prior_scale', 0.005, 5),
        'changepoint_range': trial.suggest_float('changepoint_range', 0.8, 0.9),
        'seasonality_prior_scale': trial.suggest_float('seasonality_prior_scale', 0.1, 10),
        'holidays_prior_scale': trial.suggest_float('holidays_prior_scale', 0.1, 10),
        'seasonality_mode': trial.suggest_categorical('seasonality_mode', ['multiplicative', 'additive']),
        'growth': trial.suggest_categorical('growth', ['linear', 'logistic']),
        'weekly_seasonality': trial.suggest_int('weekly_seasonality', 5, 10),
        'yearly_seasonality': trial.suggest_int('yearly_seasonality', 1, 20)
    }
    
    m = Prophet(**params)
    m.fit(train_df)
    preds = m.predict(val_df[['ds','cap','floor']])
    
    mae_score = mean_absolute_error(val_df['y'], preds['yhat'])
    
    return mae_score

In [16]:
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=10)

params = study.best_params

In [26]:
val_df.columns

Index(['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Holiday_Flag',
       'Weekly_Sales', 'Date', 'Store', 'time_idx', 'week', 'ds', 'y', 'cap',
       'floor'],
      dtype='object')

In [34]:
m = Prophet(**params)

m.fit(train_df)
val_preds = m.predict(val_df[['ds','cap','floor']])
val_preds = val_preds[['ds','yhat']]
val_real = val_df[['y','Store', 'Date']]
val_preds = pd.merge(val_preds, val_real, left_on='ds', right_on='Date', how='inner')

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [35]:
val_preds

,ds,yhat,y,Store,Date
0,2012-08-06,1.693640e+06,1697231.000,1,2012-08-06
1,2012-08-17,1.539877e+06,1597868.000,1,2012-08-17
2,2012-08-24,1.500598e+06,1494122.375,1,2012-08-24
3,2012-08-31,1.549477e+06,1582083.375,1,2012-08-31
4,2012-09-03,1.732741e+06,1675431.125,1,2012-09-03
5,2012-09-14,1.512471e+06,1517428.875,1,2012-09-14
6,2012-09-21,1.423097e+06,1506126.000,1,2012-09-21


In [37]:
mean_absolute_error(val_preds['y'], val_preds['yhat'])

35137.21151032688

In [17]:
m = Prophet(**params)

concat_train_df = pd.concat([train_df, val_df])
m.fit(concat_train_df)
preds = m.predict(test_df[['ds','cap','floor']])
preds

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds,trend,cap,yhat_lower,yhat_upper,trend_lower,trend_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,2012-09-28,1.656197e+06,2387950.25,1.268152e+06,1.432973e+06,1.656197e+06,1.656197e+06,-0.186506,-0.186506,-0.186506,-0.036221,-0.036221,-0.036221,-0.150285,-0.150285,-0.150285,0.0,0.0,0.0,1.347306e+06
1,2012-10-02,1.659120e+06,2387950.25,1.377697e+06,1.530134e+06,1.657913e+06,1.659532e+06,-0.122695,-0.122695,-0.122695,0.006617,0.006617,0.006617,-0.129312,-0.129312,-0.129312,0.0,0.0,0.0,1.455554e+06
2,2012-10-08,1.663505e+06,2387950.25,1.634259e+06,1.795228e+06,1.659258e+06,1.667268e+06,0.029698,0.029698,0.029698,0.061345,0.061345,0.061345,-0.031647,-0.031647,-0.031647,0.0,0.0,0.0,1.712907e+06
3,2012-10-19,1.671543e+06,2387950.25,1.328079e+06,1.493689e+06,1.656696e+06,1.683671e+06,-0.155951,-0.155951,-0.155951,-0.036221,-0.036221,-0.036221,-0.119730,-0.119730,-0.119730,0.0,0.0,0.0,1.410864e+06
4,2012-10-26,1.676658e+06,2387950.25,1.298942e+06,1.469457e+06,1.653961e+06,1.696165e+06,-0.172243,-0.172243,-0.172243,-0.036221,-0.036221,-0.036221,-0.136021,-0.136021,-0.136021,0.0,0.0,0.0,1.387866e+06
5,2012-11-05,1.683965e+06,2387950.25,1.618416e+06,1.799038e+06,1.648574e+06,1.715471e+06,0.014218,0.014218,0.014218,0.061345,0.061345,0.061345,-0.047127,-0.047127,-0.047127,0.0,0.0,0.0,1.707907e+06
6,2012-12-10,1.709541e+06,2387950.25,1.481940e+06,1.728018e+06,1.617257e+06,1.801744e+06,-0.062294,-0.062294,-0.062294,0.061345,0.061345,0.061345,-0.123639,-0.123639,-0.123639,0.0,0.0,0.0,1.603047e+06


In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
pd.options.mode.chained_assignment = None
test_df.loc[:, 'label'] = 'true'
test_df.loc[:, 'yhat'] = test_df.loc[:, 'y']
preds.loc[:, 'label'] = 'pred'
df_result = pd.concat([test_df[['yhat', 'label', 'ds']], preds[['yhat', 'label', 'ds']]])
fig, ax = plt.subplots(figsize=(16, 8))
sns.lineplot(y='yhat', x='ds', ax=ax, data=df_result, hue='label', markers=True)
plt.show()